In [1]:
# Pipeline for converting raw CICFlowmeter data into model friendly format

import os
import pandas as pd
import numpy as np

from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_recall_fscore_support, confusion_matrix, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from joblib import dump, load
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# df_target = pd.read_csv('cleaned_df_multilabel.csv')

df = pd.read_csv('test_cicflowmeter.csv')
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,130.68.81.1-110.209.6.55-22-47659-6,130.68.81.1,22,110.209.6.55,47659,6,13/03/2010 11:04:27 PM,102,1,1,6.0,6.0,6.0,6.0,6.000000,0.000000,6.0,6.0,6.0,0.000000,117647.058824,19607.843137,1.020000e+02,0.000000,102.0,102.0,0.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1,0,0,0,20,20,9803.921569,9803.921569,6.0,6.0,6.000,0.000000,0.000000,2,0,0,1,2,0,0,0,1.0,9.000000,6.000000,6.0,0,0,0,0,0,0,0,3,0,3,3308,7504,0,20,0,0,0,0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,No Label
1,130.68.81.1-110.209.6.55-22-47654-6,130.68.81.1,22,110.209.6.55,47654,6,13/03/2010 11:04:28 PM,203,1,1,6.0,6.0,6.0,6.0,6.000000,0.000000,6.0,6.0,6.0,0.000000,59113.300493,9852.216749,2.030000e+02,0.000000,203.0,203.0,0.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1,0,0,0,20,20,4926.108374,4926.108374,6.0,6.0,6.000,0.000000,0.000000,2,0,0,1,2,0,0,0,1.0,9.000000,6.000000,6.0,0,0,0,0,0,0,0,3,0,3,3308,7504,0,20,0,0,0,0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,No Label
2,110.209.6.55-130.68.81.1-47742-22-6,110.209.6.55,47742,130.68.81.1,22,6,13/03/2010 11:04:27 PM,1540226,8,7,554.0,602.0,152.0,6.0,69.250000,57.757498,448.0,6.0,86.0,161.567323,750.539207,9.738831,1.100161e+05,142460.175606,449927.0,148.0,1540226.0,2.200323e+05,228857.790673,690494.0,4444.0,1083261.0,180543.500000,159970.731601,426165.0,647.0,1,0,0,0,160,140,5.194043,4.544788,6.0,448.0,81.750,111.441165,12419.133333,1,0,0,11,15,0,0,0,0.0,87.200000,69.250000,86.0,0,0,0,0,15,750,0,36,0,40,6432,3576,7,20,0,0,0,0,1.268521e+15,328678.024715,1.268521e+15,1.268521e+15,No Label
3,110.209.6.55-130.68.81.1-47743-22-6,110.209.6.55,47743,130.68.81.1,22,6,13/03/2010 11:04:27 PM,1764365,7,8,538.0,608.0,152.0,6.0,76.857143,57.894814,448.0,6.0,76.0,152.232904,649.525467,8.501642,1.260261e+05,131634.311263,423219.0,148.0,1764365.0,2.940608e+05,168083.910067,630333.0,187802.0,1494048.0,213435.428571,123834.008718,425468.0,653.0,1,0,0,0,140,160,3.967433,4.534209,6.0,448.0,81.125,111.861745,12513.050000,1,0,0,10,15,0,0,0,1.0,86.533333,76.857143,76.0,0,0,0,0,15,649,0,35,0,40,6432,3576,6,20,0,0,0,0,1.268521e+15,478188.668392,1.268521e+15,1.268521e+15,No Label
4,130.68.81.1-110.209.6.55-22-47690-6,130.68.81.1,22,110.209.6.55,47690,6,13/03/2010 11:04:27 PM,1877031,2,0,12.0,0.0,6.0,6.0,6.000000,0.000000,0.0,0.0,0.0,0.000000,6.393075,1.065513,1.877031e+06,0.000000,1877031.0,1877031.0,1877031.0,1.877031e+06,0.000000,1877031.0,1877031.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,40,0,1.065513,0.000000,6.0,6.0,6.000,0.000000,0.000000,1,0,0,1,2,0,0,0,0.0,9.000000,6.000000,0.0,0,0,0,0,0,0,1,6,0,0,3308,0,1,20,0,0,0,0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,No Label


In [2]:
# print(df_target.columns)
# len(df_target.columns)

In [3]:
# print(df.columns)

In [4]:
# df.isnull().sum()  

In [5]:
# Reengineer ports
df['is_well_known_port'] = df.apply(lambda row: 1 if row['Dst Port'] < 1024 else 0, axis=1)
df['is_registered_port'] = df.apply(lambda row: 1 if row['Dst Port'] >= 1024 and row['Dst Port'] < 49152 else 0, axis=1)

# Drop certain columns

cols_to_drop = ['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
                'Timestamp', 'Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg',
                'Fwd Blk Rate Avg', 'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Label']

df_out = df.drop(columns=cols_to_drop, axis=1)

# Reorganize column order


# Scale columns


In [6]:
len(df_out.columns)

70

In [7]:
print(df_out.columns)
# print(df_target.columns)

Index(['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Fwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
       'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Pkts',
       'Subflow F

In [9]:
scaler = load('std_scaler_cicids17.bin')

cols = list(df_out.columns.values)
cols.remove('Fwd PSH Flags')
cols.remove('Fwd URG Flags')
cols.remove('FIN Flag Cnt')
cols.remove('SYN Flag Cnt')
cols.remove('RST Flag Cnt')
cols.remove('PSH Flag Cnt')
cols.remove('ACK Flag Cnt')
cols.remove('URG Flag Cnt')
cols.remove('CWE Flag Count')
cols.remove('ECE Flag Cnt')
cols.remove('is_well_known_port')
cols.remove('is_registered_port')

df_out[cols] = scaler.transform(df_out[cols])


In [10]:
df_out.head(20)

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is_well_known_port,is_registered_port
0,-0.437925,-0.135398,-0.104502,-0.058703,-0.066122,-0.267229,-0.185646,-0.262595,-0.236066,-0.457627,-0.455412,-0.502297,-0.417016,-0.049169,-0.197388,-0.296468,-0.372311,-0.379811,-0.052809,-0.429381,-0.279023,-0.348693,-0.373250,-0.112173,-0.328592,-0.194973,-0.227930,-0.261221,-0.109704,1,0,-0.152543,-0.129812,-0.212646,0.061117,-0.351718,-0.476615,-0.541352,-0.474696,-0.321141,2,0,0,1,2,0,0,0,0.442664,-0.544869,-0.262595,-0.502297,-0.164858,-0.059053,-0.128228,-0.066190,-0.283397,0.708412,-0.113221,-0.927239,-0.132918,-0.095793,-0.150497,-0.112577,-3.543230e-01,-0.123503,-3.602235e-01,-3.378490e-01,0,1
1,-0.437922,-0.135398,-0.104502,-0.058703,-0.066122,-0.267229,-0.185646,-0.262595,-0.236066,-0.457627,-0.455412,-0.502297,-0.417016,-0.051423,-0.237091,-0.296445,-0.372311,-0.379806,-0.052773,-0.429381,-0.279023,-0.348693,-0.373250,-0.112173,-0.328592,-0.194973,-0.227930,-0.261221,-0.109704,1,0,-0.152543,-0.129812,-0.233060,-0.068006,-0.351718,-0.476615,-0.541352,-0.474696,-0.321141,2,0,0,1,2,0,0,0,0.442664,-0.544869,-0.262595,-0.502297,-0.164858,-0.059053,-0.128228,-0.066190,-0.283397,0.708412,-0.113221,-0.927239,-0.132918,-0.095793,-0.150497,-0.112577,-3.543230e-01,-0.123503,-3.602235e-01,-3.378490e-01,0,1
2,-0.391271,0.070822,0.037854,0.005264,-0.058449,-0.054085,-0.185646,0.091411,-0.018936,-0.259332,-0.455412,-0.385902,-0.253377,-0.053671,-0.277147,-0.271266,-0.354514,-0.361637,-0.052792,-0.382624,-0.254842,-0.325260,-0.345429,-0.111624,-0.288964,-0.173470,-0.201897,-0.235556,-0.109621,1,0,0.014849,-0.002918,-0.253654,-0.198287,-0.351718,-0.283590,-0.320905,-0.322347,-0.315057,1,0,0,11,15,0,0,0,-0.973550,-0.337355,0.091411,-0.385902,-0.164858,-0.055201,-0.128228,-0.065714,-0.061662,0.218266,0.161828,-0.927239,-0.132918,-0.095793,-0.150497,-0.112577,5.350684e+07,-0.063326,5.135095e+07,5.432972e+07,1,0
3,-0.384481,0.041362,0.061580,0.003396,-0.058371,-0.054085,-0.185646,0.133988,-0.018420,-0.259332,-0.455412,-0.400452,-0.262831,-0.053675,-0.277152,-0.267596,-0.355866,-0.362716,-0.052792,-0.375819,-0.246707,-0.331483,-0.347853,-0.088978,-0.273936,-0.169552,-0.207778,-0.235598,-0.109621,1,0,-0.009064,0.018231,-0.253660,-0.198288,-0.351718,-0.283590,-0.322724,-0.321772,-0.315011,1,0,0,10,15,0,0,0,0.442664,-0.339125,0.133988,-0.400452,-0.164858,-0.055318,-0.128228,-0.065714,-0.061662,0.218266,0.122535,-0.927239,-0.132918,-0.095793,-0.150497,-0.112577,5.350684e+07,-0.035952,5.135095e+07,5.432972e+07,1,0
4,-0.381068,-0.105938,-0.128228,-0.058003,-0.066200,-0.267229,-0.185646,-0.262595,-0.236066,-0.460318,-0.545053,-0.511027,-0.417016,-0.053699,-0.277182,0.133884,-0.372311,-0.303981,0.606708,-0.372399,-0.072747,-0.348693,-0.297622,0.119662,-0.328592,-0.194973,-0.227930,-0.261221,-0.109704,0,0,-0.128630,-0.150961,-0.253672,-0.198408,-0.351718,-0.476615,-0.541352,-0.474696,-0.321141,1,0,0,1,2,0,0,0,-0.973550,-0.544869,-0.262595,-0.511027,-0.135398,-0.058703,-0.128228,-0.066229,-0.283397,-0.227956,-0.073928,-0.927239,-0.132918,-0.095793,-0.150497,-0.112577,-3.543230e-01,-0.123503,